# STEP 0 Check GPU  

## 本笔记本必须在GPU模式下运行，可以运行上方的nvidia-smi查看GPU是否装载。若没有，点击左上角>修改>笔记本设置，把硬件加速改成GPU。  

## This notebook must be run in GPU mode. You can run the nvidia-smi command above to check if the GPU is loaded. If not, click on the top-left corner > Edit > Notebook settings, and change the hardware accelerator to GPU.  






In [ ]:
!nvidia-smi

Mon Oct 30 13:47:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title STEP 1 Clone repository
!git clone https://github.com/LIYUESEN/druggpt.git
!pwd
import os
os.chdir("/content/druggpt")
!pwd

Cloning into 'druggpt'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 95 (delta 26), reused 5 (delta 4), pack-reused 56
Receiving objects: 100% (95/95), 50.74 KiB | 1.75 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content
/content/druggpt


In [ ]:
#@title STEP 2 Build environment
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda config --set always_yes yes --set changeps1 no
!conda update -q conda

!conda create -y -n druggpt python=3.7
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

!source activate druggpt && pip install --root-user-action=ignore datasets transformers scipy scikit-learn
!source activate druggpt && pip install --root-user-action=ignore torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
!source activate druggpt && conda install -c openbabel openbabel -y
!source activate druggpt && conda list

--2023-10-30 13:47:09--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120771089 (115M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 115.18M   148MB/s    in 0.8s    

2023-10-30 13:47:10 (148 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [120771089/120771089]

PREFIX=/usr/local
Unpacking payload ...
                                                                             
Installing base environment...





Preparing transaction: - \ | / - done
Executing transaction: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
installation finished.
    You currently have a PYTHONPATH environme

In [ ]:
#@title STEP 3 Run druggpt

#@markdown # **Instructions**
#@markdown - If you select "protein amino acid sequence", input a protein amino acid sequence in the input_protein_amino_acid_sequence field.
#@markdown - If you select "fasta file", first upload the fasta file to the druggpt directory by dragging and dropping it into the file browser on the left side. Then, input the name of the fasta file as text in the input_fasta_file field.
#@markdown - If "no input" is selected, the input content will be ignored.

#@markdown #step 3.1 Select your input type
Input_type = "fasta file" #@param ["protein amino acid sequence", "fasta file", "no input"]
#@markdown #step 3.2 Provide the required input based on your selected input type:

#@markdown - For "protein amino acid sequence", input a protein amino acid sequence.
input_protein_amino_acid_sequence = "MAKQPSDVSSECDREGRQLQPAERPPQLRPGAPTSLQTEPQGNPEGNHGGEGDSCPHGSPQGPLAPPASPGPFATRSPLFIFMRRSSLLSRSSSGYFSFDTDRSPAPMSCDKSTQTPSPPCQAFNHYLSAMASMRQAEPADMRPEIWIAQELRRIGDEFNAYYARRVFLNNYQAAEDHPRMVILRLLRYIVRLVWRMH" #@param {type:"raw"}
#@markdown - Upload the fasta file to the "druggpt" directory by selecting the file and uploading it. Ensure that the file is placed within the same folder as the default "bcl2.fasta" file. Then, input the name of the fasta file in the text field.
input_fasta_flie = "bcl2.fasta" #@param {type:"raw"}
if Input_type == "protein amino acid sequence" :
  input_content = input_protein_amino_acid_sequence
elif Input_type == "fasta file":
  input_content = input_fasta_flie
else:pass




#@markdown #step 3.3 Choose whether to use ligand prompt
ligand_prompt = False #@param {type:"boolean"}
#@markdown -  If ligand prompt is not checked, the ligand prompt content will be ignored.
if ligand_prompt == True:
  ligand_prompt_content = "COc1ccc(cc1)C(=O)" #@param {type:"raw"}
  ligand_content = "\'" + ligand_prompt_content + "\'"


#@markdown #step 3.4 Set Additional Parameters

#@markdown - At least how many molecules will be generated.
number = 50 #@param {type:"integer"}

#@markdown - Hardware device to use. Default is 'cuda'.
device = "cuda" #@param ["cuda", "cpu"]

#@markdown - Output directory for generated molecules. Default is './ligand_output/'.
output = './ligand_output/' #@param {type:"raw"}

#@markdown - how many molecules will be generated per batch. Try to reduce this value if you have low RAM. Default is 32.
batch_size = 32 #@param {type:"integer"}

#@markdown # Don't forget to run the cell.
temp = "source activate druggpt && TOKENIZERS_PARALLELISM=false && python drug_generator.py "
if Input_type == "no input":
  temp = temp + "-e "
elif Input_type == "protein amino acid sequence":
  temp = temp + "-p " + input_content + " "
else:
  temp = temp + "-f " + input_content + " "

if (ligand_prompt == True) and (Input_type != "no input"):
  temp = temp + "-l " + ligand_content + " "

temp = temp + "-n " + str(number) + " -d " + device + " -o " +output + " -b "+ str(batch_size)
print(temp)

!{temp}



source activate druggpt && TOKENIZERS_PARALLELISM=false && python drug_generator.py -f bcl2.fasta -n 50 -d cuda -o ./ligand_output/ -b 32

  _____                    _____ _____ _______ 
 |  __ \                  / ____|  __ \__   __|
 | |  | |_ __ _   _  __ _| |  __| |__) | | |   
 | |  | | '__| | | |/ _` | | |_ |  ___/  | |   
 | |__| | |  | |_| | (_| | |__| | |      | |   
 |_____/|_|   \__,_|\__, |\_____|_|      |_|   
                     __/ |                     
                    |___/                      
 A generative drug design model based on GPT2
    
<|startoftext|><P>MAKQPSDVSSECDREGRQLQPAERPPQLRPGAPTSLQTEPQGNPEGNHGGEGDSCPHGSPQGPLAPPASPGPFATRSPLFIFMRRSSLLSRSSSGYFSFDTDRSPAPMSCDKSTQTPSPPCQAFNHYLSAMASMRQAEPADMRPEIWIAQELRRIGDEFNAYYARRVFLNNYQAAEDHPRMVILRLLRYIVRLVWRMH<L>
=====Batch 1=====
Generating ligand SMILES ...
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

In [ ]:
#@title STEP 4 Save results
#@markdown # Enter the name of the results folder you want to download:
from google.colab import files
results_dir_name = 'ligand_output' #@param {type:"raw"}
results_zip = results_dir_name + ".zip"
print(results_zip)
!zip -r $results_zip $results_dir_name
files.download(results_zip)

ligand_output.zip
updating: ligand_output/ (stored 0%)
updating: ligand_output/087da39a80337e1ecae136591057cc5a550def2a.sdf (deflated 79%)
updating: ligand_output/c1ab01d8d8065065223ea79a1f97d62ee29c0908.sdf (deflated 79%)
updating: ligand_output/f4835628f8ae917711294fcd4c1571cc6fbed460.sdf (deflated 80%)
updating: ligand_output/594df3fb11f7094c2cec04e8ebf90be1ef1cc059.sdf (deflated 80%)
updating: ligand_output/57c6a15e69be09079f22792df9152288f78def6a.sdf (deflated 79%)
updating: ligand_output/3e0fec9a499f3838c66c0f3491752603fb145c66.sdf (deflated 80%)
updating: ligand_output/47f78d04501fc3fcea8a6e0daa68f2b2f6c53879.sdf (deflated 80%)
updating: ligand_output/d40635579dd313f5024f3eedd753cedb45f87ff9.sdf (deflated 80%)
updating: ligand_output/d0470e10f620110f274a7313066af1a8c05ca95b.sdf (deflated 79%)
updating: ligand_output/46fc299e8868e68e53cece213353a72021a76466.sdf (deflated 80%)
updating: ligand_output/39f18ab2ca588bb86509de7ba54ddc23432efd67.sdf (deflated 80%)
updating: ligand_outp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Step 5: Visualize and post-process on your local computer  

## 在完成模型预测和下载结果之后，您可以在本地计算机上对结果进行可视化和后处理。以下是一些建议的步骤：

## 1. 解压下载的结果文件夹：找到您在 Step 4 中下载的 zip 文件，然后将其解压缩以访问结果文件。

## 2. 安装分子可视化软件：在进行可视化之前，请确保您已经安装了分子可视化软件，如 PyMOL、UCSF Chimera 或 Discovery Studio Visualizer 等。这些工具可以帮助您更直观地查看蛋白质结构和配体。

## 3. 打开蛋白质和配体结构：使用分子可视化软件打开蛋白质结构文件（例如 PDB 格式）和下载结果中的配体结构文件（例如 SDF 格式）。

## 4. 分析预测结果：在可视化蛋白质-配体复合物时，关注以下几点：

- 配体与蛋白质的结合位置和姿态  
- 配体与蛋白质之间的氢键、疏水相互作用和其他相互作用  
- 任何可能影响结合的构象变化  
- 结果与已知实验数据（如晶体结构、结合测定等）的一致性  

## 5. 进一步后处理：根据需要，您可以使用其他生物信息学工具（如 AutoDock、GROMACS 或 Amber 等）对预测结果进行能量最小化、动力学模拟或其他分析。

## 6. 分享结果：如果您对分析结果满意，可以将您的发现与同事、合作者或社区分享。这可以通过撰写报告、创建演示文稿或发布在线资源来实现。




## After completing the model predictions and downloading the results, you can visualize and post-process the results on your local computer. Here are some suggested steps:

## 1. Unzip the downloaded results folder: Locate the zip file you downloaded in Step 4 and unzip it to access the result files.

## 2. Install molecular visualization software: Before visualizing, make sure you have installed molecular visualization software, such as PyMOL, UCSF Chimera, or Discovery Studio Visualizer. These tools can help you visualize the protein structures and ligands more intuitively.

## 3. Open the protein and ligand structures: Use the molecular visualization software to open the protein structure file (e.g., in PDB format) and the ligand structure files from the downloaded results (e.g., in SDF format).

## 4. Analyze the predicted results: When visualizing the protein-ligand complexes, pay attention to the following aspects:

- The binding position and pose of the ligand with the protein  
- Hydrogen bonds, hydrophobic interactions, and other interactions between the ligand and protein  
- Any conformational changes that might affect binding  
- The consistency of the results with known experimental data (e.g., crystal structures, binding assays, etc.)

## 5. Further post-processing: As needed, you can use other bioinformatics tools (e.g., AutoDock, GROMACS, or Amber) to perform energy minimization, molecular dynamics simulations, or other analyses on the predicted results.

## 6. Share the results: If you are satisfied with the analysis results, you can share your findings with colleagues, collaborators, or the community. This can be done by writing reports, creating presentations, or publishing online resources.